<a href="https://colab.research.google.com/github/vaibhavgp69/nlp-basics/blob/main/Sequential_Backward_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from sklearn.base import clone
from itertools import combinations
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [3]:
# k_features parameter to spicify the desired number of features wanted to return  , transform function returns the new data array with selected feature columns

class SBS():
  def __init__(self,estimator,k_features,scoring=accuracy_score,test_size=0.25,random_state=10):
    self.scoring=scoring
    self.estimtor=clone(estimator)
    self.k_features=k_features
    self.test_size=test_size
    self.random_state=random_state
  def fit(self,X,y):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=self.test_size,random_state=self.random_state)
    dim=X_train.shape[1]
    self.indices_=tuple(range(dim))
    self.subsets_=[self.indices_]
    score=self._calc_score(X_train,y_train,X_test,y_test,self.indices_)
    self.scores=[score]
    while dim>self.k_features:
      scores= []
      subsets=[]
      for p in combinations(self.indices_,r=dim-1):
        score=self._calc_score(X_train,y_train,X_test,y_test,p)
        scores.append(score)
        subsets.append(p)
      best=np.argmax(scores)
      self.indices_=subsets[best]
      self.subsets_.append(self.indices_)
      dim-=1
      self.scores_.append(scores[best])
    self.k_score_=self.scores_[-1]

    return self


  def transform(self,X):
    return X[:,self.indices_]

  def _calc_score(self,X_train,y_train,X_test,y_test,indices):
    self.estimator.fit(X_train[:,indices],y_train)
    y_pred=self.estimator.predict(X_test[:,indices])
    score=self.scoring(y_test,y_pred)
    return score
                    

In [ ]:
# THIS CODE BELOW TO USE IT IN YOUR MODELS TO REDUCE DIMENSIONS ( also plot and anaylse if working)
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighours=5)
sbs=SBS(knn,k_features=1)
sbs.fit(X_train,y_train)
k_feat=[len(k) for k in sbs.subsets_]
plt.plot(k_feat,sbs.scores_,marker='o')
plt.ylin([0.7 , 1.1])
plt.ylabel('Accuracy')
plt.xlabel('Number of features')
plt.grid()
plt.show()

